# Combining Pre-Built & Custom AI Services

In this notebook, you will integrate with the Text Analytics API to augment the claims processing capabilities. In the end, you will integrate the API calls to the summarizer and classifier services that your deployed and produce a finished claim report that shows all of the processing applied to the claim text.

The Text Analytics API is a cloud-based service that provides Natural Language Processing (NLP) features for text mining and text analysis. Here we will look at the following features from the API:

- Sentiment Analysis
- Opinion mining
- Key phrase extraction
- Language detection
- PII detection

### Setup helper functions

Run the cell below to enable helper functions to save locally the outputs as pickle files from the various Text Analytics services.

In [ ]:
import pickle
import os

output_location = './cs_outputs'
os.makedirs(output_location, exist_ok=True)

def save_output(output, name):
    file_name = os.path.join(output_location, name) + '.pkl'
    with open(file_name, 'wb') as f:
        pickle.dump(output, f, pickle.HIGHEST_PROTOCOL)
        
def get_output(file_path):
    with open(file_path, 'rb') as f:
        return pickle.load(f)

## Setup

Update the following cell with the correct **endpoint URL** and **key** for your deployed instance of the Text Analytics API and run the cell. Be sure your `endpoint URL` value ends in a slash (/).

In [ ]:
endpoint = '...' #"<your_text_analytics_api_endpoint>"
key = '...' #"<your_text_analytics_key>"

**Instantiate the Text Analytics Client**

In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient

credential = AzureKeyCredential(key)
client = TextAnalyticsClient(endpoint=endpoint, credential=credential)

**Setup the example claims document**

In [ ]:
claim = ("I had called earlier to report a car accident and I spoke with Jane and she was very helpful. "
         "However, the wait time on the call was unacceptable, I was on it for more than 30 minutes. "
         "As requested, my license plate number is ABC2021. "
         "Like I said on the phone, the accident was the other SUV fault "
         "for making a sharp turn and hitting my car on the right passenger side. "
         "Thankfully I was not hurt but the damage to the car is substantial. "
         "I request you to process the claim urgently and give me a loner vehicle for the duration of repairs. "
         "My mobile phone is 55-999-5555 if you need to reach me. Thank you.")
print(claim)

## Sentiment Analysis

The Text Analytics API's Sentiment Analysis feature is used for detecting positive and negative sentiment. If you send a Sentiment Analysis request, the API will return sentiment labels (such as "negative", "neutral" and "positive") and confidence scores at the sentence and document-level.

Run the cell below and observe the following:
- Overall document level sentiment with breakdown of the sentiment scores
- Sentence level sentiment with breakdown of the sentiment scores

In the end we will save the raw response from the **analyze_sentiment** API in local directory: `output_location`.

In [ ]:
response = client.analyze_sentiment(documents=[claim])[0]
overall_sentiment = response.sentiment
print("Document Sentiment: {}".format(overall_sentiment))
overall_positive_score = response.confidence_scores.positive
overall_neutral_score = response.confidence_scores.neutral
overall_negative_score = response.confidence_scores.negative
print("Overall scores: positive={0:.2f}; neutral={1:.2f}; negative={2:.2f} \n".format
      (overall_positive_score, overall_neutral_score, overall_negative_score))
for idx, sentence in enumerate(response.sentences):
    print("Sentence: {}".format(sentence.text))
    print("Sentiment: {}".format(sentence.sentiment))
    print("Sentence score: Positive={0:.2f} Neutral={1:.2f} Negative={2:.2f}\n".format
          (sentence.confidence_scores.positive, 
           sentence.confidence_scores.neutral, 
           sentence.confidence_scores.negative
          )
         )
save_output(response, 'sentiment_results')

## Opinion Mining

You can also use the **analyze_sentiment** API to mine opinions in the document. Opinion Mining provides granular information about the opinions related to words in the text. 

Run the below cell and review the extracted opinions from claim document. Observe that the API detects the opinion expressed by the user regarding wait times.

In [ ]:
opinions = []
response = client.analyze_sentiment(documents=[claim], show_opinion_mining=True)[0]
for sentence in response.sentences:
    for mined_opinion in sentence.mined_opinions:
        opinion = {}
        opinion['Sentence'] = sentence.text
        print("Sentence: {}".format(sentence.text))
        target = mined_opinion.target
        opinion['target'] = target.text
        opinion['target_sentiment'] = target.sentiment
        print("Target: '{}' Sentiment: {} (scores: Positive={} Negative={})".format(
            target.text, 
            target.sentiment, 
            round(target.confidence_scores.positive, 1), 
            round(target.confidence_scores.negative, 1)))
        opinion['assessments'] = []
        for assessment in mined_opinion.assessments:
            item = {}
            item['assessment'] = assessment.text
            item['assessment_sentiment'] = assessment.sentiment
            opinion['assessments'].append(item)
            print("Assessment: '{}' Sentiment: {} (scores: Positive={} Negative={}".format(
                assessment.text, 
                assessment.sentiment,
                round(assessment.confidence_scores.positive, 1), 
                round(assessment.confidence_scores.negative, 1)))
        print("\n")
        opinions.append(opinion)
save_output(response, 'opinion_mining_results')

## Key Phrase Extraction

The Key Phrase Extraction (KPE) capability of the Text Analytics API is useful to quickly identify the main points in a collection of documents. We will apply KPE to our claims text to extract main concepts in the text. Run the cell below.

In [ ]:
key_phrases = []
try:
    response = client.extract_key_phrases(documents = [claim])[0]
    if not response.is_error:
        print("\tKey Phrases:")
        for phrase in response.key_phrases:
            key_phrases.append(phrase)
            print("\t\t", phrase)
        save_output(response, 'kpe_results')
    else:
        print(response.id, response.error)
except Exception as err:
    print("Encountered exception. {}".format(err))

## Language Detection

The Language Detection feature of the Azure Text Analytics REST API evaluates text input for each document and returns language identifiers with a score that indicates the strength of the analysis.

First, we will translate our claim to Spanish language and then evaluate the translated claim. Run the following two cells to detect the claims language.

In [ ]:
claim_spanish = ("Había llamado antes para reportar un accidente automovilístico y hablé con Jane "
                 "y ella fue de gran ayuda. Sin embargo, el tiempo de espera de la llamada fue inaceptable, "
                 "estuve en ella durante más de 30 minutos. Según lo solicitado, mi número de placa es ABC2021. "
                 "Como dije por teléfono, el accidente fue la otra falla de la SUV por hacer un giro brusco y "
                 "golpear mi auto en el lado derecho del pasajero. Afortunadamente no me lastimé, pero el daño "
                 "al auto es sustancial. Le solicito que procese el reclamo con urgencia y me dé un vehículo "
                 "solitario mientras dure la reparación. Mi teléfono móvil es 55-999-5555 si necesita "
                 "comunicarse conmigo. Gracias.")
print(claim_spanish)

In [ ]:
try:
    response = client.detect_language(documents = [claim_spanish], country_hint = 'us')[0]
    print("Language:", response.primary_language.name)
    print("Confidence Score:", response.primary_language.confidence_score)
    save_output(response, 'language_detection_results')
except Exception as err:
    print("Encountered exception. {}".format(err))

Let's detect the language of the original claim.

In [ ]:
language = None
try:
    response = client.detect_language(documents = [claim], country_hint = 'us')[0]
    language = response.primary_language.name
    print("Language:", response.primary_language.name)
    print("Confidence Score:", response.primary_language.confidence_score)
    save_output(response, 'language_detection_results')
except Exception as err:
    print("Encountered exception. {}".format(err))

## Personally Identifiable Information (PII) Detection

The PII Detection extracts personal information from an input text and gives you the option of masking it. Run the cell below to identify PII in the claims text.

In [ ]:
from azure.ai.textanalytics import PiiEntityCategory
redacted_claim = None
response = client.recognize_pii_entities([claim], language="en", 
                                         categories_filter=[PiiEntityCategory.PERSON, 
                                                            PiiEntityCategory.PHONE_NUMBER])
result = [doc for doc in response if not doc.is_error]
for doc in result:
    redacted_claim = doc.redacted_text
    print("Redacted Text: {}".format(redacted_claim))
    for entity in doc.entities:
        print("Entity: {}".format(entity.text))
        print("\tCategory: {}".format(entity.category))
        print("\tConfidence Score: {}".format(entity.confidence_score))
    save_output(response, 'pii_detection_results')

## Save the Results in Blob Store

Save the JSON responses that came from the various cognitive services to a permanent store like the blob storage for future reference.

We have been saving the JSON outputs as pickle files temporarily in a local directory. Let’s review the saved files.

In [ ]:
!ls $output_location

**Create the `Workspace` from the saved config file**

In [ ]:
from azureml.core import Workspace
ws = Workspace.from_config()

**Upload the files from the local directory to the default blob store**

In [ ]:
datastore = ws.get_default_datastore()
datastore.upload(output_location, 
                 target_path = 'cs_results', 
                 overwrite = True, 
                 show_progress = True)

Navigate to your resource group’s storage account, select **Containers**, and then select the default blob store **azureml-blobstore-xxx**. Next, select the folder **cs_results** to view the saved files.


![Image showing uploaded results from the cognitive services APIs.](./images/blob_files.png 'Blobs')

## Invoking the Azure ML Deployed Services

Run the following cell to define a method that will be used to invoke your classifier and summarizer methods deployed using Azure Machine Learning service to Azure Container Instances:

In [ ]:
import requests
def invoke_service(ml_service_key, ml_service_scoring_endpoint, ml_service_input):
    headers   = {"Authorization": "Bearer " + ml_service_key}
    response  = requests.post(ml_service_scoring_endpoint, headers=headers, json=ml_service_input)
    result = response.json()
    return result

Configure the classifier invocation with the key and endpoint URI (**classifier_service_scoring_endpoint**) as appropriate to your deployed instance:

> This is the scoring endpoint URI you copied from the notebook named `04 Deploy Classifier Web Service.ipynb`.

In [ ]:
classifier_service_key = "" #leave this value empty if the service does not have authentication enabled
#"<your_classifier_scoring_url>"
classifier_service_scoring_endpoint = '...'
classifier_service_input = [claim]

Invoke the classifier and observe the result:

In [ ]:
classifier_result = invoke_service(classifier_service_key, 
                                   classifier_service_scoring_endpoint, classifier_service_input)
classifier_result

In [ ]:
# Interpret the classifier result
classification = 'Auto Insurance Claim' if classifier_result == 1 else 'Home Insurance Claim' 
classification

Similarly, configure the key and scoring endpoint URI (**summarizer_service_scoring_endpoint**) as appropriate to your summarizer service:

> This is the scoring endpoint URI you copied from the notebook named `02 Deploy Summarizer Web Service.ipynb`.

In [ ]:
summarizer_service_key = "" #leave this value empty if the service does not have authentication enabled
#"<your_summarizer_service_url>"
summarizer_service_scoring_endpoint = '...'
summarizer_service_input = claim

Invoke the summarizer service and observe the result:

In [ ]:
summarizer_result = invoke_service(summarizer_service_key, summarizer_service_scoring_endpoint, 
                                   summarizer_service_input)
formatted_summary =  summarizer_result[0].replace("\\n", " ").strip() if len(summarizer_result) > 0 else "N/A"
print(formatted_summary)

## Summarizing the Results

In this final task, you pull together all of the pieces to display the results of your AI based processing.

Run the following cell and examine the result.

In [ ]:
from IPython.core.display import HTML

displayTemplate = """
<div><h1>Claim Summary</h1></div>
<div>&nbsp;</div>
<div>Claim Type: <b>{}</b></div>
<div>Language: {}</div>
<div>Overall Sentiment: {}</div>
<div>Overall Positive Sentiment Score: {}</div>
<div>Overall Negative Sentiment Score: {}</div>
<div>&nbsp;</div>
<div>Key Phrases:</div>
<div><b>{}</b></div>
<div><b>{}</b></div>
<div><b>{}</b></div>
<div><b>{}</b></div>
<div><b>{}</b></div>
<div>&nbsp;</div>
<div>Key Opinion:</div>
<div><pre>{}</pre></div>
<div>Target: <b>{}</b> Sentiment: <b>{}</b></div>
<div>Assessment: <b>{}</b> Sentiment: <b>{}</b></div>
<div>&nbsp;</div>
<div>Claim Summary:</div>
<div><pre>{}</pre></div>
<div>&nbsp;</div>
<div>Redacted Claim:</div>
<div>{}</div>
"""
displayTemplate = displayTemplate.format(classification, 
                                         language, 
                                         overall_sentiment, 
                                         overall_positive_score, 
                                         overall_negative_score,  
                                         key_phrases[0],
                                         key_phrases[1],
                                         key_phrases[2],
                                         key_phrases[3],
                                         key_phrases[4],
                                         opinions[0]['Sentence'],
                                         opinions[0]['target'],
                                         opinions[0]['target_sentiment'], 
                                         opinions[0]['assessments'][0]['assessment'],
                                         opinions[0]['assessments'][0]['assessment_sentiment'],
                                         formatted_summary, 
                                         redacted_claim
                                        )
display(HTML(displayTemplate))